In [1]:
import re
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import pathlib

In [2]:
def define_trend(row):
    if row[4] >= row[5] and row[5] >= row[6]:
        trend = 'Aging_Hypo'
    elif row[4] <= row[5] and row[5] <= row[6]:
        trend = 'Aging_Hyper'
    else:trend = 'Differ_in_Aging'
    return trend

In [3]:
#test_celltype = 'Oligo_NN'
_dir = '/home/qzeng/project/aging/230807_call_dmr/dss'

In [4]:
conditions = ['Aging_Hypo','Aging_Hyper','Differ_in_Aging']

# generate Hypo Hyper Bed files

In [5]:
dmr_paths = glob.glob(f'{_dir}/mC_FILTER_DMR_BED/*.DMR.bed')

In [6]:
for path in dmr_paths:
    df = pd.read_csv(path, on_bad_lines='skip', sep = '\t', header = None,index_col = 3)
    mt = pathlib.Path(path).name.split('.')[0]
    if mt not in ['Oligo_NN','SNc-VTA-RAmb_Foxa1_Dopa','STR-PAL_Chst9_Gaba','SUB-ProS_Glut']:
        df[4] = round(df[4],1)
        df[5] = round(df[5],1)
        df[6] = round(df[6],1)

        df['trend'] = df.apply(define_trend, axis = 1)

        p = pathlib.Path(f"condition_dmr/{mt}")
        p.mkdir(parents=True, exist_ok=True)

        for condition in conditions:
            tmpdf = df[df['trend'] == condition].copy()[[0,1,2]]
            tmpdf.to_csv(f"condition_dmr/{mt}/{condition}.DMR.bed", 
                         sep = '\t',header = None, index = None)      
    

# Homer

In [7]:
wkdir = os.getcwd()
bedfiles = glob.glob(f"condition_dmr/*/*.bed")

In [8]:
motif_fh = open("runHomerMotif.sh", 'w')
for bedfile in bedfiles:
    cell_type = bedfile.split("/")[-2]
    prefix = re.sub(".bed", "", os.path.basename(bedfile))
    outdir = os.path.dirname(bedfile)
    
    print(f"findMotifsGenome.pl {bedfile} mm10 {outdir}/motif_{prefix} -size 200 &", file=motif_fh)
motif_fh.close()

In [16]:
!pwd

/home/qzeng/project/aging/230819-motif-analysis
